MNISTをpytorchで自力で実装するだけのプログラム 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

google colab 用に使用
URLのhttps://github.com を https://colab.research.google.com/github に変換する

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

In [ ]:
mnist_data = MNIST('~/tmp/mnist', train=True, download=True, transform=transforms.ToTensor())
data_loader = DataLoader(mnist_data,batch_size=4,shuffle=False)
data_iterator = iter(data_loader)

images, labels = data_iterator.next()
data = images[2].numpy()
plt.imshow(data.reshape(28, 28), cmap='inferno', interpolation='bicubic')
plt.show()
print('ラベル:', labels[0])

とりあえず中身の可視化

In [ ]:
# 学習データ
train_data_with_label = MNIST('~/tmp/mnist', train=True, download=True, transform=transforms.ToTensor())
train_data_loader = DataLoader(train_data_with_label,batch_size=4,shuffle=True)
# 検証データ
test_data_with_label = MNIST('~/tmp/mnist', train=False, download=True, transform=transforms.ToTensor())
test_data_loader = DataLoader(test_data_with_label,batch_size=4,shuffle=False)

In [ ]:
class Flatten(nn.Module): # なんで無いの
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

model = nn.Sequential()
model.add_module('input', nn.Conv2d(in_channels=1, out_channels= 32, kernel_size=(3, 3))) # 28* 28 *1 to 26 * 26 * 32
model.add_module('fc1', nn.Conv2d(in_channels = 32, out_channels = 64,kernel_size = (3,3))) # 26* 26 * 32 to 24 * 24 * 64
model.add_module('relu', nn.ReLU())
model.add_module('fc2', nn.MaxPool2d(kernel_size = 2)) #  12 * 12 * 64
model.add_module('Dropout',nn.Dropout(p=0.25))
modek.add_module('FC1',nn.Linear(12 * 12 * 64,128))
model.add_module('relu2', nn.ReLU())
model.add_module('Dropout',nn.Dropout(p=0.5))
modek.add_module('FC2',nn.Linear(12 * 12 * 64,10))
model.add_module('softmax',nn.Softmax())

In [ ]:
from torch import optim
 
# 誤差関数の設定
loss_fn = nn.CrossEntropyLoss()  # 変数名にはcriterionも使われる
 
# 重みを学習する際の最適化手法の選択
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 5. 学習と推論の設定
# 5-1. 学習1回でやることを定義します
# Chainerのtraining.Trainer()に対応するものはない
 
from torch.autograd import Variable
 
 
def train(epoch):
    model.train()  # ネットワークを学習モードに切り替える
 
    # データローダーから1ミニバッチずつ取り出して計算する
    for data, target in loader_train:
        data, target = Variable(data), Variable(target)  # 微分可能に変換
        optimizer.zero_grad()  # 一度計算された勾配結果を0にリセット
 
        output = model(data)  # 入力dataをinputし、出力を求める
        loss = loss_fn(output, target)  # 出力と訓練データの正解との誤差を求める
        loss.backward()  # 誤差のバックプロパゲーションを求める
        optimizer.step()  # バックプロパゲーションの値で重みを更新する
 
    print("epoch{}：終了\n".format(epoch))

In [ ]:
# 5. 学習と推論の設定
# 5-2. 推論1回でやることを定義します
# Chainerのtrainer.extend(extensions.Evaluator())に対応するものはない
 
 
def test():
    model.eval()  # ネットワークを推論モードに切り替える
    correct = 0
 
    # データローダーから1ミニバッチずつ取り出して計算する
    for data, target in loader_test:
        data, target = Variable(data), Variable(target)  # 微分可能に変換
        output = model(data)  # 入力dataをinputし、出力を求める
 
        # 推論する
        pred = output.data.max(1, keepdim=True)[1]  # 出力ラベルを求める
        correct += pred.eq(target.data.view_as(pred)).sum()  # 正解と一緒だったらカウントアップ
 
    # 正解率を出力
    data_num = len(loader_test.dataset)  # データの総数
    print('\nテストデータの正解率: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))